# Use GPIO library to read input: Ultrasonic Sensor
In this exercise, we try to use GPIO library to read the input data presented by the ultrasonic sensor.


## Turn off bluetooth_service
Before we start, ensure you turn off bluetooth_control service on the RPi car. The service is there for your Android app to communicate actions to remote control the car. We do not need that now. Leaving it on would interfere with the GPIO exercises here. Open a terminal from Jupyter and type the command below after the dollar sign.
```bash
$ sudo systemctl stop bluetooth_control
```

## Code cells
You will find plenty of code cells that contain Python code to execute on the RPi car. A Jupyter server runs on the RPi car and receives the code from these cells in browser for the car to execute it locally on the RPi.

Follow each of the code cells below, some cells contain complete code you can run, while others contain incomplete code. You can always modify or add to the code in order to experiment.

Simply press "shift enter" together to execute the code.

## Comments
Python uses \# to provide comments that can illustrate what a programmer intends the code to do. You should read comments to help you understand the code and write comments to help readers and yourself make sense of your own code.

## Mechanism behind the ultrasonic sensor
Read the class slides for a more visual introduction.

The code cell below uses ultrasonic sensor to measure the distance between the sensor and an obstacle.

The code has all the details in comments. Here is a summary how the sensor is used:
1. Set up the trigger pin as output, and the echo pin as input
2. Send a voltage pulse (lasting longer than 10us) to the trigger pin. The unit will start transmitting the 8-cycle 40kHz ultrasonic pulses upon the trigger pulse.
3. As the ultrasonic pulses leave the sensor and start traveling, the echo pin will read HIGH until,
4. the ultrasonic pulses are bounced back and received by the sensor again, when the echo pin will read LOW again
5. the time lapse is measured for the duration in which the echo pin reads HIGH.
6. the time lapse is then converted with the sound speed of 343 meters/second into the distance that the ultrasonic waves traveled. Then the result is divided by 2 to get the distance between the sensor and the obstacle. 

Now you try to run the code. <font color=red>Be sure that when you test the code, the obstacle has a reasonably reflective surface, that is, it should not be fabric or other sound-proofing materials that can absorb ultrasonic waves, such as your pants or a curtain</font>

In [7]:
import RPi.GPIO as GPIO
import time

GPIO.setmode(GPIO.BCM)
GPIO.setwarnings(False)

# Yahboom uses broadcom pin #0 for echo input, and pin #1 for trigger signal output
PIN_ECHO = 0
PIN_TRIG = 1

# Even though we use ultrasonic sensor as an input, we still need to output an voltage pulse
# onto the trigger pin so that the unit can be triggered to transmit 8 ultrasonic pulses.
# We therefore need to set up the trigger pin as an output.
GPIO.setup(PIN_TRIG, GPIO.OUT)
# Set up the echo pin as an input
GPIO.setup(PIN_ECHO, GPIO.IN)

try:
    while True:
        distance = 0
        
        # First, we send a trigger voltage pulse to the trigger pin that lasts longer than 10us
        
        # Set the initial low for 50ms:                     :_________50ms_________
        GPIO.output(PIN_TRIG, GPIO.LOW)
        time.sleep(0.05)
        #                                                                           _15us_
        # Set the pulse high for 15us:                      :_________50ms_________| 
        GPIO.output(PIN_TRIG, GPIO.HIGH)
        time.sleep(0.000015)
        #                                                                           _15us_
        # Bring down to low again to make the trigger pulse: _________50ms_________|      |___
        GPIO.output(PIN_TRIG, GPIO.LOW)

        
        # Trigger pulse has been just sent.
        # Now the unit is starting to send 8-cycle 40kHz ultrasonic pulses
        
        
        # By now the 8-cycle ultrasonic 40kHz pulses should not have finished transmitting,
        # at this moment the echo pin should really have a LOW reading.
        if GPIO.input(PIN_ECHO) == GPIO.HIGH:
            print("ECHO pin should not be HIGH as the 8-cycle 40kHz pulses are still being sent. Aborting.")
            break
            
        # We will now try to keep track of
        #    * when the ultrasonic pulses leaves the unit:            time_start
        #    * when the ultrasonic pulses echo came back to the unit: time_stop
        
        time_start = time.time()
        time_stop = time_start

        while GPIO.input(PIN_ECHO) == GPIO.LOW:
            # as long as the 8-cycle ultrasonic pulses haven't finished transmitting
            # the echo pin should have a LOW reading,
            # we will keep setting time_start to the most current time
            # until the echo pin reads HIGH, i.e., when the 8-cycle ultrasonic pulses have just been sent
            time_start = time.time()
            if time_start - time_stop > 0.02:
                # it has taken too long and the ultrasonic pulses still haven't finished transmitting
                # this is invalid. We will abort.
                distance = -3 # set it to a negative number to indicate abort.
                break

        if distance < 0:
            print("Ultrasonic pulse taking too long to send. Aborting.")
            break

        # By now, the 8-cycle pulses have been transmitted
        # The echo pin should continue to read HIGH until echo is received, when it's set to LOW again.
        time_stop = time.time()
        while GPIO.input(PIN_ECHO) == GPIO.HIGH:
            # As long as echo pin reads high, the unit has not received the echo yet,
            # set time_stop to the current time
            time_stop = time.time()
            # if the duration between start and stop is more than 0.02 (roughly 3.43m)
            # then we will cut off at 3.43m
            if time_stop - time_start > 0.02:
                break

        # Distance formula, sound speed is set to 343 meter / second
        distance = ((time_stop - time_start) * 343 / 2) * 100 # this is calculated in centimeters
        print("distance = {cm:0.2f}cm, {inch:0.2f}inches".format(cm=distance, inch=distance*0.393701), end='\r', flush=True)
        
except KeyboardInterrupt:
    pass
finally:
    GPIO.cleanup()

print('\nTest stopped.')

distance = 343.07cm, 135.07inches
Test stopped.


## Challenge
The code above is entirely working. You are tasked below to 
1. Take what's needed from the above code and convert it into a function
```python
def distance():
    """
    Get distance between the ultrasonic sensor and an obstacle. 
    
    Returns
    -------
      - a positive number that denotes the distance in centimeters
      - if any error occurs, return a negative number
    """
    pass
```
2. Use the ultrasonic distance sensing function you just made above and have it work with LED or even DC motors output devices. Use your imagination.

In [5]:
def distance():
    """
    Get distance between the ultrasonic sensor and an obstacle. 
    
    Returns
    -------
      - a positive number that denotes the distance in centimeters
      - if any error occurs, return a negative number
    """
    # TODO study the code cell above and make a function that returns the distance measured
    # Hint: remove the while loop, and where the code "break"s, replace "break" with "return"
    pass

# TODO have LED or DC motors work with the distance sensed from distance() function defined above